<a href="https://colab.research.google.com/github/Renita1206/Image-Classification-Models/blob/main/Chest_X_rays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up the Dataset

In [ ]:
! pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d tolgadincer/labeled-chest-xray-images

In [ ]:
!unzip labeled-chest-xray-images.zip -d dataset

# Model Training

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
training_path="/content/dataset/chest_xray/train"
test_path="/content/dataset/chest_xray/test"

In [ ]:
train_dataset=tf.keras.preprocessing.image_dataset_from_directory(training_path,image_size=(200,200),batch_size=32)

In [ ]:
test_dataset=tf.keras.preprocessing.image_dataset_from_directory(test_path,image_size=(200,200),batch_size=32)

In [ ]:
CNNmodel = tf.keras.models.Sequential()
#first conv layer
CNNmodel.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200, 3)))
CNNmodel.add(tf.keras.layers.MaxPooling2D(2,2))
CNNmodel.add(tf.keras.layers.BatchNormalization())
#second conv layer
CNNmodel.add(tf.keras.layers.Conv2D(64, (3, 3), activation= 'relu'))
CNNmodel.add(tf.keras.layers.MaxPooling2D(2,2))
CNNmodel.add(tf.keras.layers.Dropout(0.3))
#third conv layer
CNNmodel.add(tf.keras.layers.Conv2D(32, (3, 3), activation= 'relu'))
CNNmodel.add(tf.keras.layers.MaxPooling2D(2,2))
#fifth conv layer
CNNmodel.add(tf.keras.layers.Conv2D(128, (3, 3), activation= 'relu'))
CNNmodel.add(tf.keras.layers.MaxPooling2D(2,2))
CNNmodel.add(tf.keras.layers.Dropout(0.4))

# Neural Network - For classification
CNNmodel.add(tf.keras.layers.Flatten())
CNNmodel.add(tf.keras.layers.Dense(64, activation='relu'))
CNNmodel.add(tf.keras.layers.Dropout(0.4))
CNNmodel.add(tf.keras.layers.Dense(16, activation='relu'))
CNNmodel.add(tf.keras.layers.Dropout(0.3))
CNNmodel.add(tf.keras.layers.Dense(4,activation='softmax'))

In [ ]:
CNNmodel.summary()

In [ ]:
CNNmodel.compile(optimizer='adam',
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics = ['accuracy'])

In [ ]:
history = CNNmodel.fit(train_dataset, validation_data=test_dataset, epochs = 8)

# Graphs

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.ylim([0.5,1])
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model')
plt.xlabel('epoch')
plt.ylabel('loss')
# plt.ylim([0.5,1])
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Output Part

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

In [ ]:
#from google.colab import files
#files.upload()

In [ ]:
img_path = "/content/P2.jpg"
img = image.load_img(img_path, target_size=(200, 200, 3))
plt.imshow(img)
plt.show()

In [ ]:
img_array = image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = CNNmodel.predict(img_array)
score = tf.nn.softmax(predictions[0])

In [ ]:
class_names = train_dataset.class_names
print(class_names)

In [ ]:
#print(decode_predictions(pred, top=3)[0])
result=np.sort(score)
result_name=np.argsort(score)


In [ ]:
print("Patient Diagnosis: ",class_names[result_name[-1]])